# Train Models

In [2]:
# imports
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.utils.validation import check_array, check_X_y

In [376]:
# wrapper class, implemented with xgboost docs
class XGBTrainEarlyStoppingClassifier(BaseEstimator, ClassifierMixin):
    def __init__(
        self,
        n_estimators=10000,
        early_stopping_rounds=50,
        validation_fraction=0.1,
        random_state=0,
        n_jobs=1,
        max_depth=3,
        learning_rate=0.1,
        subsample=1.0,
        colsample_bytree=1.0,
        reg_lambda=1.0,
        min_child_weight=1.0,
        gamma=0.0):
        self.n_estimators = n_estimators
        self.early_stopping_rounds = early_stopping_rounds
        self.validation_fraction = validation_fraction

        self.random_state = random_state
        self.n_jobs = n_jobs

        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.reg_lambda = reg_lambda
        self.min_child_weight = min_child_weight
        self.gamma = gamma

    def fit(self, X, y):
        X, y = check_X_y(X, y, accept_sparse=True)
        X_tr, X_val, y_tr, y_val = train_test_split(
            X, y,
            test_size=self.validation_fraction,
            random_state=self.random_state,
            stratify=y)

        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        dval = xgb.DMatrix(X_val, label=y_val)

        params = {
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "seed": self.random_state,
            "nthread": self.n_jobs,
            "max_depth": self.max_depth,
            "eta": self.learning_rate,
            "subsample": self.subsample,
            "colsample_bytree": self.colsample_bytree,
            "lambda": self.reg_lambda,
            "min_child_weight": self.min_child_weight,
            "gamma": self.gamma,
            "verbosity": 0}

        self.booster_ = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=self.n_estimators,
            evals=[(dval, "val")],
            early_stopping_rounds=self.early_stopping_rounds,
            verbose_eval=False,
        )

        # best_ntree_limit
        self.best_ntree_limit_ = getattr(self.booster_, "best_ntree_limit", None)
        self.best_iteration_ = getattr(self.booster_, "best_iteration", None)
        return self

    def _pred_kwargs(self):
        if self.best_ntree_limit_ is not None:
            return {"ntree_limit": self.best_ntree_limit_}
        if self.best_iteration_ is not None:
            return {"iteration_range": (0, self.best_iteration_ + 1)}
        return {}

    def predict_proba(self, X):
        X = check_array(X, accept_sparse=True)
        d = xgb.DMatrix(X)
        p = self.booster_.predict(d, **self._pred_kwargs())
        p = np.asarray(p).ravel()
        return np.column_stack([1.0 - p, p])

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


In [378]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# final csv
full_csv = pd.read_csv('../data/processed/s9_final.csv')

majority class: 1


In [380]:
# the target var
y = full_csv['outcome']
X = full_csv.drop(columns=['outcome'])


print(X.shape)
print(y.shape)

(514, 571)
(514,)


In [381]:
# collect features

# splitting by numeric and cat features. note: all features with numbers are num in this dataset
def split_num_cat_by_value(X: pd.DataFrame):
    coerced = X.apply(lambda s: pd.to_numeric(s, errors="coerce"))
    is_numeric_col = coerced.notna().sum(axis=0).eq(X.notna().sum(axis=0))

    num_ftrs = X.columns[is_numeric_col].tolist()
    cat_ftrs = X.columns[~is_numeric_col].tolist()
    
    return num_ftrs, cat_ftrs

num_ftrs, cat_ftrs = split_num_cat_by_value(X)

# no ordinal ftrs in this dataset
ordinal_ftrs = []
ordinal_cats = []

# no overlapping features
assert set(num_ftrs) & set(cat_ftrs) == set()

In [382]:
test = X.drop(columns=cat_ftrs)
cols_with_missing = test.columns[test.isna().any()].tolist()

print(cols_with_missing)

bad_rows = test.index[test[cols_with_missing].isna().any(axis=1)]
print("num na rows:", len(bad_rows))
print("na row indices:", bad_rows.tolist()[:50])

['ttk_ds1_to_atk_str']
num na rows: 3
na row indices: [85, 86, 87]


In [383]:
# fill any non-attacking unit with high ttk
X["ttk_ds1_to_atk_str"] = X["ttk_ds1_to_atk_str"].fillna(1e6)

len(X.columns)

571

In [384]:
print(X.shape)
print(y.shape)

(514, 571)
(514,)


In [385]:
random_states = [0,1,2,3,5]

<h3>Baselines</h3>

In [386]:
def eval_from_proba(y_true, y_proba, thr=0.5):
    y_hat = (y_proba >= thr).astype(int)
    
    return {
        "acc": accuracy_score(y_true, y_hat),
        "roc_auc": roc_auc_score(y_true, y_proba),
        "precision": precision_score(y_true, y_hat, zero_division=0),
        "recall": recall_score(y_true, y_hat, zero_division=0),
        "f1": f1_score(y_true, y_hat, zero_division=0),
    }

def baseline_over_seeds(X, y, random_states, test_size=0.1):
    rows = []

    for r in random_states:
        _, _, y_other, y_test = train_test_split(X, y, test_size=test_size, random_state=r, stratify=y)
        p = float(np.mean(y_other))
        y_proba = np.full(shape=len(y_test), fill_value=p, dtype=float)
        m = eval_from_proba(y_test, y_proba)

        rows.append({"seed": r, "cv_acc": 1-p, **{f"test_{k}": v for k, v in m.items()}}) # here majority is class 0
    
    out = pd.DataFrame(rows)
    summary = out.drop(columns=["seed"]).agg(["mean", "std"])
    return out, summary

baseline_df, baseline_summary = baseline_over_seeds(X, y, random_states)
print(baseline_summary)
baseline_summary.to_csv('../results/baseline.csv')

        cv_acc  test_acc  test_roc_auc  test_precision  test_recall  test_f1
mean  0.521645  0.519231           0.5             0.0          0.0      0.0
std   0.000000  0.000000           0.0             0.0          0.0      0.0


for each random state:
    initiate the parameter grid for these five models
    - logistic regression, elastic net
    - random forest, svc, xgboost, knn
    train 5 models over this random state

In [387]:
# make the preprocessors for data
def make_preprocessor(num_ftrs, cat_ftrs) -> ColumnTransformer:
    # define pipelines
    numeric_transformer = Pipeline(steps=[
        ('scalar', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        # fill nans with missing, treating as separate cat
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'))])

    return ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_ftrs),
            ('cat', categorical_transformer, cat_ftrs)])

In [388]:
na_counts = X[num_ftrs].isna().sum().sort_values(ascending=False)
print(na_counts[na_counts > 0].head(100))


Series([], dtype: int64)


In [389]:
for r in random_states:
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.1, random_state=r, stratify=y)

    prep = make_preprocessor(num_ftrs, cat_ftrs)
    prep.fit(X_other)

    Xt_other = prep.transform(X_other)
    Xt_test  = prep.transform(X_test)

    print(Xt_other.shape)
    print(Xt_test.shape)

    assert np.isnan(Xt_other).any() == False
    assert np.isnan(Xt_test).any() == False

(462, 865)
(52, 865)
(462, 865)
(52, 865)
(462, 864)
(52, 864)
(462, 866)
(52, 866)
(462, 865)
(52, 865)


<h3>Make pipelines and grids</h3>

In [390]:
# function to make models n grids
def make_models_and_grids(random_state: int) -> tuple:
    models = {
        'logreg_l2': LogisticRegression(max_iter=10000, random_state=random_state, penalty='l2', solver='lbfgs'),
        'logreg_elastic': LogisticRegression(max_iter=10000, random_state=random_state, penalty='elasticnet', solver='saga', n_jobs=3),
        'rfc': RandomForestClassifier(n_jobs=3, random_state=random_state, n_estimators=500),
        'svc': SVC(kernel='rbf', probability=True, random_state=random_state),
        'knn': KNeighborsClassifier(n_jobs=3),
        'xgb': XGBTrainEarlyStoppingClassifier(random_state=random_state, n_estimators=10000, early_stopping_rounds=50,
                validation_fraction=0.1, n_jobs=1)}

    grids = {
        "logreg_l2": {
            "clf__C": [0.0001, 0.001, 0.01, 0.1, 1, 10],
            "clf__class_weight": [None, "balanced"],
        },

        "logreg_elastic": {
            "clf__C": [0.01, 0.1, 1, 10, 100],
            "clf__l1_ratio": [0.5, 0.8, 0.9, 0.95, 1.00],
            "clf__class_weight": [None, "balanced"],
        },

        "rfc": {
            "clf__n_estimators": [400, 500, 600, 700, 800],
            "clf__max_depth": [None, 2, 3, 5, 8],
            "clf__min_samples_leaf": [3, 5, 8],
            "clf__max_features": ["sqrt", 0.25, 0.5],
            "clf__class_weight": [None, "balanced"],
        },

        "svc": {
            "clf__C": [0.1, 0.5, 1, 2, 5, 10],
            "clf__gamma": ["scale", 0.01, 0.1, 0.5, 1],
            "clf__class_weight": [None, "balanced"],
        },

        "knn": {
            "clf__n_neighbors": [1, 2, 3, 5, 9],
            "clf__weights": ["uniform", "distance"],
            "clf__p": [1, 2],
        },

        "xgb": {
            "clf__max_depth": [0, 1, 2, 3],
            "clf__learning_rate": [0.03, 0.1, 0.3, 0.6, 1],
            "clf__subsample": [0.1, 0.2, 0.3, 0.6, 1.0],
            "clf__colsample_bytree": [0.8, 0.9, 0.95, 0.98, 1.0],
            "clf__reg_lambda": [0.001, 0.01, 0.1, 1.0, 10.0],
        },
    }

    return models, grids

In [391]:
# return 1d array of pos class
def get_cont_scores(fitted_pipeline, X):
    if hasattr(fitted_pipeline, "decision_function"):
        scores = fitted_pipeline.decision_function(X)
        return np.asarray(scores).ravel()

    if hasattr(fitted_pipeline, "predict_proba"):
        proba = fitted_pipeline.predict_proba(X)
        return np.asarray(proba)[:, 1].ravel()

In [392]:
# run a model over all randoms
def one_model(X, y, model_n: str, num_ftrs, cat_ftrs, random_states, test_size=0.1, scoring='accuracy'):
    rows = []

    out_dir = Path("../results/final/grid_pipe")
    out_dir.mkdir(parents=True, exist_ok=True)

    for r in random_states:
        models, grids = make_models_and_grids(r)
        base_model = models[model_n]
        param_grid = grids[model_n]
        
        X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=test_size, random_state=r, stratify=y)

        preprocessor = make_preprocessor(num_ftrs, cat_ftrs)

        pipe = Pipeline(steps=[
            ('prep', preprocessor),
            ('clf', clone(base_model))])

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=r)

        gs = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring=scoring, cv=cv, n_jobs=-1, refit=True)
        gs.fit(X_other, y_other)

        # evaluate on held out test
        best_pipe = gs.best_estimator_
        y_pred = best_pipe.predict(X_test)
        scores = get_cont_scores(best_pipe, X_test)
        
        # label metrics
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        roc_auc_s = roc_auc_score(y_test, scores)

        rows.append({
            "seed": r,
            'model': model_n,
            'best_cv_acc': gs.best_score_,
            'test_acc': accuracy_score(y_test, y_pred),
            'test_roc_auc': roc_auc_s,
            'best_params': gs.best_params_,
            "test_precision": prec,
            "test_recall": rec,
            "test_f1": f1})
        
        # save grid info
        pd.DataFrame(gs.cv_results_).to_csv(out_dir / f"{model_n}_seed{r}_cv_results.csv", index=False)
        with open(out_dir/f"{model_n}_seed{r}_best_pipe.pkl", "wb") as f:
            pickle.dump(best_pipe, f)
        with open(out_dir/f"{model_n}_seed{r}_X_test.pkl", "wb") as f:
            pickle.dump(X_test, f)
        with open(out_dir/f"{model_n}_seed{r}_y_test.pkl", "wb") as f:
            pickle.dump(y_test, f)
        
        print(f'done with random state {r}')

    print(f'done with model {model_n}')
    
    # saving results
    out = pd.DataFrame(rows)
    metric_cols = ["best_cv_acc", "test_acc", "test_precision", "test_recall", "test_f1","test_roc_auc"]
    summary = out[metric_cols].agg(['mean','std']).T
    summary = summary.reset_index().rename(columns={'index': 'metric'})

    return out, summary

<h3>Train</h3>

In [1]:
models = ['logreg_l2', 'logreg_elastic', 'rfc', 'svc', 'knn', 'xgb']

# 'logreg_l2', 'logreg_elastic', 'rfc', 'svc', 'knn', 'xgb'

# for m in models:
#     out, summary = one_model(
#         X=X,
#         y=y,
#         model_n=m,
#         num_ftrs=num_ftrs,
#         cat_ftrs=cat_ftrs,
#         random_states=random_states,
#         test_size=0.1,
#         scoring="accuracy")

#     out.to_csv(f'../results/final/{m}.csv', index=False)
#     summary.to_csv(f'../results/final/{m}_agg.csv', index=False)
#     print(summary)